In [ ]:
import datetime
import numpy
import pygetm

%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.Domain.create_cartesian(500.*numpy.arange(100), 500.*numpy.arange(30), 1)
domain.T.H[1:-1, 1:-1] = 50.
domain.T.mask[1:-1, 1:-1] = 1
domain.initialize()

# Idealized surface forcing
tausx, tausx_ = domain.T.array(fill=0.)
tausx[...] = 0.01
tausy_ = numpy.zeros_like(tausx)
sp, sp_ = domain.T.array(fill=0.)

advection = pygetm.Advection(domain, scheme=1)
momentum = pygetm.Momentum(domain, advection)
sealevel = pygetm.Sealevel(domain)
pressure = pygetm.Pressure(domain)

# Time
start = datetime.datetime(2020, 1, 1, 0, 0)
stop = datetime.datetime(2020, 1, 1, 2, 0)
timestep = 10.
time = start

# Set up velocity plot
nsample = 4
fig = matplotlib.pyplot.figure()
ax = fig.gca()
pc = ax.pcolormesh(domain.T.xi, domain.T.yi, domain.T.z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label('elevation (m)')
Q = ax.quiver(domain.T.x[::nsample, ::nsample], domain.T.y[::nsample, ::nsample], momentum.U[::nsample, ::nsample], momentum.V[::nsample, ::nsample], scale=.3)
title = ax.set_title(time.strftime('%H:%M:%S'))

In [ ]:
plotting_interval = 5
istep = 0
while time < stop:
    istep += 1

    momentum.advection_2d(timestep)
    pressure.surface(domain.T.z_, sp_)
    momentum.uv_momentum_2d(timestep, tausx_, tausy_, pressure.dpdx_, pressure.dpdy_)
    sealevel.update(timestep, momentum.U_, momentum.V_)
    domain.depth_update()

    # Update velocity plot
    if istep % plotting_interval == 0:
        Q.set_UVC(momentum.U[::nsample, ::nsample], momentum.V[::nsample, ::nsample])
        title.set_text(time.strftime('%H:%M:%S'))
        pc.set_array(domain.T.z.ravel())
        fig.canvas.draw()

    time += datetime.timedelta(seconds=timestep)
    